In [ ]:
%matplotlib inline
import os
import torch
import random
import pickle
import numpy as np 
import pandas as pd 
from PIL import Image
import torch.nn as nn 
from pathlib import Path
import torch.nn.functional as F
from matplotlib import pyplot as plt 
from tqdm import tqdm, tqdm_notebook
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
d = {'ImageId': df['ImageId'].unique()}
train_df = pd.DataFrame(d)
train_df['ClassesList'] = ['' for i in range(len(train_df))]
train_df['EncodedPixels'] = ['' for i in range(len(train_df))]
for i in range(len(train_df)):
    train_df['ClassesList'][i] = df.loc[df['ImageId']==train_df['ImageId'][i]]['ClassId'].values.tolist()
    train_df['EncodedPixels'][i] = df.loc[df['ImageId']==train_df['ImageId'][i]]['EncodedPixels'].values.tolist()

In [ ]:
def make_mask( encoded, shape=(1600,256)):
    # Делим на два списка в соответствии с кодировкой
    if isinstance(encoded, str):
        encoded = list(map(int, encoded.split(' ')))
#     print(encoded)
    full,pixel,number = [],[],[]
    [pixel.append(encoded[i]) if i%2==0 else number.append(encoded[i]) for i in range(0, len(encoded))]
    # "Раскрываем" кодировку, получаем индексы закрашенных пикселей
    k=0
    for i in range(len(number)):
        for j in range(number[i]):
            ind = pixel[i]+j
            full.append(ind-1)
        k +=number[i]
    # Создаем массив под готовое изображение    
    mask = np.zeros((1600*256,1), dtype=int)
    # Закрашиваем соответствующие пиксели
    mask[full] = 255
    #преобразем к размерам фотографий металла
    res = np.reshape(mask,(1600, 256)).T
    res = Image.fromarray(res.astype(np.uint8))
    return res

In [ ]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE_1 = 800
RESCALE_SIZE_2 = 128
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [ ]:
class SteelDataset(Dataset):
    def __init__(self, names, df, mode):
        super().__init__()
        self.names = names
        self.mode = mode
        if self.mode != 'test':
            self.df = df

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.names)
        
    def __len__(self):
        return self.len_
      
    def load_sample(self, file, mode):
        if mode == 'test':
            image = Image.open('../input/severstal-steel-defect-detection/test_images/'+ file).convert("RGB")
        else:
            image = Image.open('../input/severstal-steel-defect-detection/train_images/'+ file).convert("RGB")
        image.load()
        return image
    
    def __getitem__(self, index):
            transforms_tens = T.Compose([
            T.ToTensor()])

            # загружаем и меняем размер изображения
            img = self._prepare_sample(self.load_sample(self.names[index], self.mode))

            if self.mode == 'test':
                img = np.array(img)
                max_value = 256 ** ((img.dtype == np.uint16) + 1) - 1
                img = (img / max_value).astype(np.float32)
                img = transforms_tens(img)
                return img
            
            # загружаем классы и маски
            labels = list(self.df['ClassesList'].loc[self.df['ImageId'] == self.names[index]])[0]
            mask = list(self.df['EncodedPixels'].loc[self.df['ImageId'] == self.names[index]])[0]
            num_obj = len(labels)
            masks = np.zeros((RESCALE_SIZE_2, RESCALE_SIZE_1, 4), dtype=np.float32)
            masks = np.transpose(masks,(2, 0, 1))
            for i in range(4):
                for j in range(num_obj):
                    if i == (labels[j]-1):
                # раскодирование масок
                        masks[i] = np.array(self._prepare_sample(make_mask(mask[j])))
                        masks[i] = masks[i] / 255

            # преобазование в тензоры
            masks = torch.as_tensor(masks, dtype=torch.float32)
            label = torch.as_tensor(labels)
            
            # преобразуем изображения и маски
            if self.mode == "train":
                img, masks= transforms_all(img,masks)
                
            img = np.array(img)
            max_value = 256 ** ((img.dtype == np.uint16) + 1) - 1
            img = (img / max_value).astype(np.float32)
            img = transforms_tens(img)
            
            return img, masks
        
    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE_1, RESCALE_SIZE_2))
        return image

In [ ]:
def transforms_all(image, segmentation):
    if random.random() > 0.5:
        image = TF.autocontrast(image)
    if random.random() > 0.5:
        image = TF.hflip(image)
        segmentation = TF.hflip(segmentation)
    if random.random() > 0.5:
        image = TF.vflip(image)
        segmentation = TF.vflip(segmentation)
    return image, segmentation

In [ ]:
from sklearn.model_selection import train_test_split
train_val_names = train_df['ImageId']
train_files,val_files = train_test_split(train_val_names, train_size=0.75)
val_dataset = SteelDataset(list(val_files), train_df,  mode='val')
train_dataset = SteelDataset(list(train_files),train_df, mode='train')

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer, batch_size):
    model.train()
    running_loss = 0.0
    dice = 0.0
    iou = 0.0
    for X_batch, Y_batch in train_loader:
        inputs = X_batch.to(DEVICE)
        labels = Y_batch.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.detach().cpu().numpy() 
        dice += dice_metric(outputs, labels)
    train_loss = running_loss / len(train_loader)
    train_dice = dice / len(train_loader)
    return train_loss, train_dice

In [ ]:
def eval_epoch(model, val_loader, criterion, batch_size):
    model.eval()
    dice = 0.0
    iou = 0.0
    running_loss = 0.0
    for X_batch, Y_batch in val_loader:
        inputs = X_batch.to(DEVICE)
        labels = Y_batch.to(DEVICE)

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs,labels.long())
            running_loss += loss.item()
            dice += dice_metric(outputs, labels)
    val_loss = running_loss / len(val_loader)
    val_dice = dice / len(val_loader)

    return val_loss, val_dice

In [ ]:
def train(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers = 2, shuffle=True)#,collate_fn = collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,num_workers = 2, shuffle=False)#,collate_fn = collate_fn)
    history = []
    maxIoU = 0
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} train_dice: {t_dice:0.4f} \
    \nValidation  val_loss: {v_loss:0.4f} val_dice: {v_dice:0.4f} "
    criterion = nn.BCEWithLogitsLoss()
    
    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        params = model.parameters()
        opt = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(opt,
                                                   step_size=20,
                                                   gamma=0.1)
        for epoch in range(epochs):
            train_loss, train_dice = fit_epoch(model, train_loader, criterion, opt, batch_size)
            lr_scheduler.step()    
            val_loss, val_dice = eval_epoch(model, val_loader, criterion, batch_size)
            history.append((train_loss, train_dice, val_loss, val_dice))
            if val_IoU > maxIoU:
                maxIoU = val_IoU
                torch.save(model, './weight_unet.dat')
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss, t_dice = train_dice, \
                                           v_loss=val_loss, v_dice = val_acc ))   
    return history

In [ ]:
def dice_metric(probability, truth, threshold=0.5, reduction='none'):
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = truth.float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = dice_pos
    return dice

In [ ]:
class UnetConv2dBlock(nn.Module):

    def __init__(self, in_channel: int, mid_channel: int, out_channel: int, kernel_size: list = [3,3], stride_size: list = [1,1], activation: str = 'relu'):
        super().__init__()

        if activation == 'swish':
            self.conv2dblock = nn.Sequential(
            nn.Conv2d(in_channel, mid_channel, kernel_size=kernel_size[0], stride=stride_size[0], padding=1),
            nn.BatchNorm2d(mid_channel),
            Swish(),
            nn.Conv2d(mid_channel, out_channel, kernel_size=kernel_size[1], stride=stride_size[1], padding=1),
            nn.BatchNorm2d(out_channel),
            Swish(),
        )
        else:
            self.conv2dblock = nn.Sequential(
                nn.Conv2d(in_channel, mid_channel, kernel_size=kernel_size[0], stride=stride_size[0], padding=1),
                nn.BatchNorm2d(mid_channel),
                nn.ReLU(inplace=True),
                nn.Conv2d(mid_channel, out_channel, kernel_size=kernel_size[1], stride=stride_size[1], padding=1),
                nn.BatchNorm2d(out_channel),
                nn.ReLU(inplace=True)
            )

    def forward(self, x):
        return self.conv2dblock(x)



class Swish(nn.Module):

    def __init__(self, slope = 1):
        super().__init__()
        self.slope = slope # * torch.nn.Parameter(torch.ones(1))

    def forward(self, x):
        return self.slope * x * torch.sigmoid(x)



class DownSample(nn.Module):

    def __init__(self):
        super().__init__()
        self.maxpooling = nn.MaxPool2d(2)

    def forward(self, x):
        return self.maxpooling(x)



class UpSample(nn.Module):

    def __init__(self, in_channel=None, bilinear: bool = True):
        super().__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channel , in_channel // 2, kernel_size=2, stride=2)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return x

In [ ]:
class UnetPlusPlus(nn.Module):
    
    def __init__(self, in_channel, num_classes: int, deep_supervision: bool=False):
        super(UnetPlusPlus, self).__init__()
        self.num_classes = num_classes
        self.metrics = 0 
        self.deep_supervision = deep_supervision

        num_filter = [32, 64, 128, 256, 512]

        self.down = DownSample()
        self.up = UpSample()

        self.conv_block_0_0 = UnetConv2dBlock(in_channel, num_filter[0], num_filter[0])
        self.conv_block_1_0 = UnetConv2dBlock(num_filter[0], num_filter[1], num_filter[1])
        self.conv_block_2_0 = UnetConv2dBlock(num_filter[1], num_filter[2], num_filter[2])
        self.conv_block_3_0 = UnetConv2dBlock(num_filter[2], num_filter[3], num_filter[3])
        
        self.conv_block_0_1 = UnetConv2dBlock(num_filter[0]+num_filter[1], num_filter[0], num_filter[0])
        self.conv_block_1_1 = UnetConv2dBlock(num_filter[1]+num_filter[2], num_filter[1], num_filter[1])
        self.conv_block_2_1 = UnetConv2dBlock(num_filter[2]+num_filter[3], num_filter[2], num_filter[2])

        self.conv_block_0_2 = UnetConv2dBlock(2*num_filter[0]+num_filter[1], num_filter[0], num_filter[0])
        self.conv_block_1_2 = UnetConv2dBlock(2*num_filter[1]+num_filter[2], num_filter[1], num_filter[1])

        self.conv_block_0_3 = UnetConv2dBlock(3*num_filter[0]+num_filter[1], num_filter[0], num_filter[0])

        if self.deep_supervision:
            self.final1 = nn.Conv2d(num_filter[0], self.num_classes, kernel_size=1)
            self.final2 = nn.Conv2d(num_filter[0], self.num_classes, kernel_size=1)
            self.final3 = nn.Conv2d(num_filter[0], self.num_classes, kernel_size=1)
        else:
            self.final = nn.Conv2d(num_filter[0], self.num_classes, kernel_size=1)

    def forward(self, x):

        x0_0 = self.conv_block_0_0(x)

        x1_0 = self.conv_block_1_0(self.down(x0_0))
        x0_1 = self.conv_block_0_1(self.up(x1_0, x0_0))
        
        x2_0 = self.conv_block_2_0(self.down(x1_0))
        x1_1 = self.conv_block_1_1(self.up(x2_0, x1_0))
        x0_2 = self.conv_block_0_2(self.up(x1_1, torch.cat([x0_0, x0_1], 1)))

        x3_0 = self.conv_block_3_0(self.down(x2_0))
        x2_1 = self.conv_block_2_1(self.up(x3_0, x2_0))
        x1_2 = self.conv_block_1_2(self.up(x2_1, torch.cat([x1_0, x1_1],1)))
        x0_3 = self.conv_block_0_3(self.up(x1_2, torch.cat([x0_0, x0_1, x0_2], 1)))

        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            return [output1, output2, output3]

        else:
            out = self.final(x0_3)
            return out

In [ ]:
model = UnetPlusPlus(3,4).to(DEVICE)

In [ ]:
history = train(train_dataset, val_dataset, model, epochs = 53, batch_size = 5)

In [ ]:
train_loss, train_acc, val_loss, val_acc = zip(*history)

In [ ]:
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,13))
    ax1.plot(train_loss, label='train', marker='o')
    ax1.plot(val_loss, label='val', marker='o')
    ax1.set_title('Loss per epoch')
    ax1.set_ylabel('loss');
    ax1.set_xlabel('epoch')
    ax1.legend(), ax1.grid()
    
    ax2.plot(train_dice, label='train_dice', marker='*')
    ax2.plot(val_dice, label='val_dice',  marker='*')
    ax2.set_title('Score per epoch')
    ax2.set_ylabel('mean dice')
    ax2.set_xlabel('epoch')
    ax2.legend(), ax2.grid()
    
#     plt.savefig('0.710615.png')
    plt.show()

In [ ]:
def predict_image_mask_miou(model, image, mask):
    model.eval()
    
    image = image.to(DEVICE)
    mask = mask.to(DEVICE)
    with torch.no_grad():
        
        image = image.unsqueeze(0)
        mask = mask.unsqueeze(0)
        
        output = model(image)
        print(output)
        score = dice_metric(output, mask)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked, score

In [ ]:
def miou_score(model, val_dataset):
    score_iou = []
    for i in tqdm(range(len(val_dataset))):
        img, mask = val_dataset[i]
        pred_mask, score = predict_image_mask_miou(model, img, mask)
        score_iou.append(score)
    return score_iou

In [ ]:
# model = torch.load('../input/unet-weight/weight_unet_wew.dat')

In [ ]:
print(np.mean(miou_score(model, val_dataset)))